In [1]:
import pandas as pd
df = pd.read_csv('kw.csv', delimiter='|')
df.head()

,text,source_kw,key_words
0,В Беларуси предъявили новое обвинение россияни...,"общество, беларусь, уголовное дело, егор дудни...","арест, уголовное дело, беларусь, фильм, актер,..."
1,В Калининградской области с 8 ноября до 1 февр...,"калининградская область, коронавирус, QR коды","коронавирус, калининградская область, ограниче..."
2,Южноафриканский писатель Дэймон Гэлгут стал об...,"премия, букер","писатель, букеровская премия, литература, роман,"
3,В 97-этажной башне «Федерация» в Москве-Сити к...,"криптовалюты, обмен, отмывание денег, bloomber...","киберпреступники, криптовалюта, москва сити, о..."
4,Полиция Латвии в рамках своего расследования «...,"дело магнитского, латвия","латвия, магнитский, дело магнитского, недвижим..."


Все источники взяты с сайта https://novayagazeta.ru/news. В источнике ключевые слова сделаны метками с хештегом под каждой новостью. Метки кликабельные и отображают новости с этой же меткой.

In [2]:
import RAKE
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Дарья\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
count_intersection = 0
count_non_intersection = 0
for i in range(df.shape[0]):
    for key_w in df.loc[i]["key_words"].split("\n"):
        if key_w in df.loc[i]["source_kw"].split("\n"):
            count_intersection += 1
        else:
            count_non_intersection += 1

In [4]:
print("пересечение: ", count_intersection)
print("непересечение: ", count_non_intersection)
print("процент пересечений: ", round(count_intersection / (count_intersection + count_non_intersection), 3))

пересечение:  0
непересечение:  13
процент пересечений:  0.0


(пересечения посчитаны неверно)

In [5]:
standard = [[], [], [], [], []]

In [6]:
for i in range(df.shape[0]):
    words = df["source_kw"].tolist()
    my_words = df["key_words"].tolist()
    words_to_add = []
    words_to_add.extend(words[i].split("\n"))
    words_to_add.extend(my_words[i].split("\n"))
    standard[i].append(set(words_to_add))

IndexError: list index out of range

## Rake ##

In [7]:
rake = RAKE.Rake(stop)

In [8]:
df.at[0, "rake_key_words"] = ''

In [9]:
for i in range(df.shape[0]):
    rake_pairs = rake.run(df.loc[i]["text"], maxWords=3, minFrequency=2)
    rake_kw = ""
    for pair in rake_pairs:
        rake_kw = rake_kw + pair[0] + "\n"
    df.at[i, "rake_key_words"] = rake_kw[:-1]

## TextRank ##

In [10]:
from gensim.summarization import keywords as kw

In [11]:
df.at[0, "TRank_key_words"] = ''

In [12]:
for i in range(df.shape[0]):
    df.at[i, "TRank_key_words"] = kw(df.loc[i]["text"], pos_filter=[])

## td-idf ##

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
df.at[0, "tfidf_key_words"] = ''

In [15]:
tfidf_vectorizer = TfidfVectorizer()

In [16]:
texts = df["text"]
tfidf = tfidf_vectorizer.fit_transform(texts)
names = tfidf_vectorizer.get_feature_names()
data = tfidf.todense().tolist()
tfidf_df = pd.DataFrame(data, columns=names)

In [17]:
tfidf_df = tfidf_df[filter(lambda x: x not in stop , tfidf_df.columns)]

In [18]:
for i in tfidf_df.iterrows():
    tfidf_keys = i[1].sort_values(ascending=False)[:20].index.tolist()
    tfidf_s = ""
    for key in tfidf_keys:
        tfidf_s = tfidf_s + key + "\n"
    df.at[i[0], "tfidf_key_words"] = tfidf_s[:-1]